## tradeSeq for DE along epididymal axis (epithelial spots) in male reproductive tract

In [ ]:
#if (!require("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")

#BiocManager::install("tradeSeq")

In [ ]:
library(tradeSeq)
library(SingleCellExperiment)
library(slingshot)

In [ ]:
adata <- readRDS("/lustre/scratch126/cellgen/team292/vl6/VISIUM/epididymalaxis_epi_downsampled_cytassist.rds")
adata


In [ ]:
counts <- assay(adata, "X")

In [ ]:
set.seed(7)
pseudotime <- colData(adata)$Epididymal_Axis
cellWeights <- rep(1, 158)

In [ ]:
#icMat <- evaluateK(counts = counts, sds = SlingshotDataSet(adata), k = 3:10, 
#                   nGenes = 200, verbose = T)

In [ ]:
library(scran)

In [ ]:
libsizes <- colSums(counts)
size.factors <- libsizes/mean(libsizes)
logcounts(adata) <- log2(t(t(counts)/size.factors) + 1)
assayNames(adata)

In [ ]:
# Filter genes with non-zero biological components
getVariableGenes <- function(adata,fdr_thresh=NULL,var_thresh=0) {
  gene_var <- modelGeneVar(adata)
  genes <- getTopHVGs(gene_var, var.threshold =var_thresh ,fdr.threshold = fdr_thresh)
  return(genes)
}

In [ ]:
genes <- getVariableGenes(adata)

In [ ]:
length(genes)

In [ ]:
dim(counts[genes,])

In [ ]:
sce <- fitGAM(counts = counts[genes,], pseudotime = pseudotime, cellWeights = cellWeights, nknots = 5,
                verbose = TRUE, parallel = TRUE)

In [ ]:
table(rowData(sce)$tradeSeq$converged)

In [ ]:
adata <- adata[genes, ]

In [ ]:
adata

### ASSOCIATION TEST

A first exploration of the data analysis may consist of checking whether gene expression is associated with a particular lineage. The statistical test performed here, implemented in the associationTest function, is testing the null hypothesis that all smoother coefficients are equal to each other. This can be interpreted as testing whether the average gene expression is significantly changing along pseudotime.



In [ ]:
assoRes <- associationTest(sce)
head(assoRes)

In [ ]:
write.csv(assoRes, file = '/nfs/team292/vl6/FetalReproductiveTract/manuscript_v1/supp_tables/SupplementaryTable8_epididymis_epi_cytassist.csv')


In order to discover marker genes of the progenitor or differentiated cell population, researchers may be interested in assessing differential expression between the progenitor cell population (i.e., the starting point of a lineage) with the differentiated cell type population (i.e., the end point of a lineage). The function startVsEndTest uses a Wald test to assess the null hypothesis that the average expression at the starting point of the smoother (progenitor population) is equal to the average expression at the end point of the smoother (differentiated population). The test basically involves a comparison between two smoother coefficients for every lineage. The function startVsEndTest performs a global test across all lineages by default (i.e. it compares the start and end positions for all lineages simultaneously), but you can also assess all lineages separately by setting lineages=TRUE. Below, we adopt an omnibus test across the two lineages.

In [ ]:
assoRes = assoRes[assoRes$pvalue < 0.001,]

In [ ]:
dim(assoRes)

In [ ]:
assoRes['VIL1', ]

In [ ]:
assoRes['ESR1', ]

In [ ]:
assoRes['WNT9B', ]

In [ ]:
assoRes['GATA3', ]

In [ ]:
assoRes = assoRes[assoRes$meanLogFC > 0.5, ]

In [ ]:
dim(assoRes)

In [ ]:
assoRes = assoRes[assoRes$waldStat > 50, ]

In [ ]:
dim(assoRes)

In [ ]:
write.csv(assoRes, file = "/lustre/scratch126/cellgen/team292/vl6/VISIUM/EpididymalAxis_epi_genes.csv",
          row.names = TRUE)

In [ ]:
model_vals_df <- predictSmooth(models = sce, gene=row.names(assoRes), nPoints=100) 

In [ ]:
model_vals_mtx <- predictSmooth(models = sce, gene=row.names(assoRes), nPoints=100, tidy=FALSE)

In [ ]:
write.csv(model_vals_mtx, file = '/lustre/scratch126/cellgen/team292/vl6/VISIUM/epididymis_visium_fitted_values_tradeseq_epithelium_mesenchyme.csv')


In [ ]:
head(assoRes, 30)

In [ ]:
library(ggplot2)

In [ ]:
# Get prioritised genes based on scRNA-seq 
spatially_variable = c('CLDN2', 'KRT17', 'MUC6', 'PDK4', 'ACSL5', 'EPHA7', 'CXCL14', 'CDH6', 'SLC3A1', 'POMC', 'CPM', 'PCP4', 'GATA3', 
                       'USH1C', 'MGAM', 'SALL1', 'SMIM24', 'SPP1', 'WDR72', 'TSPAN12', 'CA2', 'CPXM2', 'MME', 'PDZK1IP1', 'TFAP2A',
                       'DPEP1', 'MSLN', 'PIK3C2G', 'ABCC3', 'SPAG11A', 'HMOX1', 'VIL1', 'NTRK2', 'NSG1', 'CDH2', 'HNF4A', 'WNT6', 'PTGDS',
                       'BICDL1', 'RASD1', 'GALNT14', 'BHMT', 'SLC2A9', 'CPLX2', 'UGT2A3', 'SLC47A1', 'GLYAT', 'ZNF385B', 'COL4A4', 
                       'BLNK', 'MFSD2A', 'TNMD', 'OTULINL', 'GRHL2', 'SLC38A11', 'F2RL1', 'MLXIPL', 'HGD', 'RNF128', 'COL4A3', 'NAPSA',
                       'OR51E2', 'LYPD1', 'GMNC', 'SLC39A5', 'GNAO1', 'PTPRZ1', 'CA12', 'PDZK1', 'CLDN10', 'ESR1', 'RHEX', 'NCMAP', 
                       'SPINK2', 'SYNDIG1', 'TENM1', 'PRDM16', 'BVES', 'HNF1A', 'TM4SF18', 'SLCO4C1', 'FRMD3', 'LRP2', 'AMN', 'KLK6',
                       'HOXD1', 'SIM2', 'GIPC2', 'SERPINA1', 'ADRA2C', 'TOX3', 'ALKAL2', 'NOS1AP', 'DNAJC22', 'TUNAR', 'SLC9A4', 
                       'RIPOR2', 'LYPD6B', 'RBM20', 'SLC22A11', 'POU5F1', 'METTL7B', 'SMIM32', 'HOGA1', 'OSBPL10', 'PRSS22', 'PAQR4',
                       'CDRT15L2', 'SLC5A9', 'KCTD8', 'CDHR5', 'IL6R', 'WNT2', 'GLYATL1', 'FMO1', 'TRPM5', 'SLC9A3', 'ACY3', 'SQOR', 
                       'CAPN12', 'CRYM', 'CFTR', 'ADM2', 'SLC30A2', 'B3GNT3', 'MARCH11', 'TM4SF4', 'MT1X', 'IFITM1', 'LGALS2', 'PRAP1',
                       'STEAP4', 'S1PR5', 'RNF183', 'FGD3', 'PLEK2', 'TFEC', 'ASGR2', 'SSTR1', 'PDZD3', 'FZD5', 'EDAR', 'AZGP1',
                       'SLC16A4', 'C1orf210', 'GRPR', 'IHH', 'ENPP3', 'SOWAHA', 'C3orf85', 'DDIT4L', 'SLC27A2')

In [ ]:
for (gene in spatially_variable) {
    print(gene)
    print(plotSmoothers(sce, counts, gene = gene, alpha = 0) + ggtitle(gene) +
                          theme(title = element_text(size = 20, face = 'bold'),
                                axis.title = element_text(size = 20),
                                axis.text = element_text(size = 20),
                                legend.text = element_text(size = 20),
                                legend.title = element_text(size = 20, face = "bold")))
}